In [1]:
#pragma cling add_include_path("/user/local/include")

In [2]:
#include "xeus-cling-includeLib.h"

In [3]:
#include <iostream>

In [4]:
#include <opencv2/opencv.hpp>

In [5]:
#include <opencv2/highgui.hpp>

In [6]:
#include <opencv2/core.hpp>

In [7]:
#include <opencv2/imgproc.hpp>

In [8]:
using namespace std;

In [9]:
using namespace cv;

## ROI의 orient 주성분을 추출해서 local dominant orient 를 계산하는 테스트

In [10]:
Mat img = imread("img/cameraman.jpg", IMREAD_GRAYSCALE);

In [31]:
Mat roi1 = img(Range(128,256), Range(0,128));
imwrite("img/roi.jpg", roi1);

In [12]:
cout << roi1.cols << " "<< roi1.rows;

128 128

#### SVD 함수에 입력할 orient 행렬(G=[gx, gy]) 생성

In [13]:
Mat sobelx, sobely;

In [14]:
Sobel(roi1, sobelx, CV_64F, 1, 0);
Sobel(roi1, sobely, CV_64F, 0, 1);

#### gx, gy 각각 flatten 하게 1xN 행렬로 변형한다.(rows=N, cols=1 인 행렬.)

In [15]:
Mat gx = sobelx.reshape(1, 1).t();

In [16]:
Mat gy = sobely.reshape(1, 1).t();

In [17]:
cout <<"gx shape: "<< gx.cols << " "<< gx.rows << endl;
cout <<"gy shape: "<< gy.cols << " "<< gy.rows;

gx shape: 1 16384
gy shape: 1 16384

#### gx, gy를 2xN 행렬로 변환.

In [18]:
int total = gx.total();

In [19]:
Mat fin_mat = Mat::zeros(Size(2, total), CV_32F);

In [20]:
for(int i=0; i<total; i++)
{
    fin_mat.at<float>(i*2) = gx.at<float>(i);
    fin_mat.at<float>(i*2+1) = gy.at<float>(i);
}

In [21]:
cout << fin_mat.rows << " " << fin_mat.cols;

16384 2

In [22]:
Mat w, u, vt;

In [23]:
Mat roi1_64f;
roi1.convertTo(roi1_64f, CV_64F);

입력 matrix 형태가 CV_64FC1 or CV_32FC1 둘 중 하나여야 한다.

In [24]:
SVD::compute(roi1_64f, w, u, vt, SVD::FULL_UV);

In [25]:
cout << w.cols <<" " <<w.rows;

1 128

In [26]:
cout << u.cols <<" "<<u.rows;

128 128

In [27]:
cout << vt.cols << " "<< vt.rows;

128 128

### SVD 결과로 나온 VT에서 dominant orientation, dominant "edge"orientation 구하기.
vt matrix=   
| v11 v12 |   
| v21 v22 | 이다. arctan(v11/v12)=dominant orientation, arctan(v21/v22)=dominant "edge" orientation.임.

In [28]:
int ddata[4] = {1,2,3,4};
Mat vt_mat(Size(2,2), CV_32S, ddata);
cout << vt_mat;

[1, 2;
 3, 4]

In [29]:
int v11 = vt_mat.at<int>(Point(0,0));
int v21 = vt_mat.at<int>(Point(1,0));
int v12 = vt_mat.at<int>(Point(0,1));
int v22 = vt_mat.at<int>(Point(1,1));

In [30]:
cout << v11 << endl;
cout << v21 << endl;
cout << v12 << endl;
cout << v22 << endl;

1
2
3
4
